In [1]:
import re
import os
import pandas as pd
import random
from tqdm import tqdm 

file_paths, file_names, emotions, audios = [], [], [], []
emotion_map = {'Neutral': 'neutral', 'Anger': 'angry', 'Happiness': 'happy', 'Sadness': 'sad', 'Fear': 'fear',
              'Disgust': 'disgust'}

main_path = '/home/rl3155/Multilingual-Speech-Emotion-Recognition-System/Spanish/Spanish_New'
DATA_NATURAL = "/home/rl3155/MESD_All"

In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1
0.12.1+cu113
cuda


In [3]:
bundle = torchaudio.pipelines.HUBERT_LARGE
extractor = bundle.get_model()
print(extractor.__class__)
print(bundle.sample_rate)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
16000


### Random get 5 parts of the entries -- only run code below 1 time

In [4]:
# Define path to datasets; create a random chunked 5 parts of entries
# entries = os.listdir(DATA_NATURAL)
# random.shuffle(entries)
# session = []
# equal_parts = (len(entries)-1)//5 # for equally split the entries into 5 parts
# count = 0
# for i in tqdm(range(len(entries))):
#     entry = entries[i]
#     if "wav" not in entry:
#         continue
#     path = DATA_NATURAL + "/" + entry
#     emotion = emotion_map[entry.split("_")[0]]
#     file_paths.append(path)
#     file_names.append(entry)
#     emotions.append(emotion)
    
#     # assign session to it
#     part = (count//equal_parts)%6 + 1
#     if part == 6:
#         part = 5
#     session.append(part)
#     count += 1

# file = pd.DataFrame({'path':file_paths, 'name': file_names, 'emotion': emotions, 'session': session})
# dataframe_path = main_path + '/session_entries.csv'
# file.to_csv(dataframe_path)

### Extract Features using Models

In [4]:
# Get the entries with assigned session

dataframe_path = '/home/rl3155/Multilingual-Speech-Emotion-Recognition-System/Spanish/session_entries.csv'
file = pd.read_csv(dataframe_path)[['path', 'name', 'emotion', 'session']]
file.head()

,path,name,emotion,session
0,/home/rl3155/MESD_All/Fear_F_A_basta_ya.wav,Fear_F_A_basta_ya.wav,fear,1
1,/home/rl3155/MESD_All/Happiness_M_A_arriba.wav,Happiness_M_A_arriba.wav,happy,1
2,/home/rl3155/MESD_All/Fear_F_B_arana.wav,Fear_F_B_arana.wav,fear,1
3,/home/rl3155/MESD_All/Neutral_M_B_articulo.wav,Neutral_M_B_articulo.wav,neutral,1
4,/home/rl3155/MESD_All/Disgust_F_A_antes.wav,Disgust_F_A_antes.wav,disgust,1


In [5]:
from tqdm import tqdm

for i in tqdm(range(len(file['path']))):
    path = file['path'][i]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    feature = [f[0] for f in feature]
    audio = torch.stack(feature)
    audios.append(audio)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 862/862 [05:55<00:00,  2.43it/s]


In [6]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, audios, labels, label_transform):
        super(MyDataSet).__init__()
        self.audios = audios
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        label = self.label_transform[self.labels[idx]]
        audio = self.audios[idx]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [7]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [8]:
categories = ['neutral', 'angry', 'happy', 'sad', 'fear', 'disgust']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'neutral': 0, 'angry': 1, 'happy': 2, 'sad': 3, 'fear': 4, 'disgust': 5}

## Train with 3CNN+LSTM

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=7):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits

### Model Traning on each layer 

In [34]:
holdout = 5
train = file[file['session'] != holdout]
train_audios = [audios[i] for i in range(len(audios)) if file['session'][i] != holdout]
test = file[file['session'] == holdout]
test_audios = [audios[i] for i in range(len(audios)) if file['session'][i] == holdout]

In [35]:
from torch.utils.data import DataLoader

train_dataset = MyDataSet(train_audios, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=16, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

test_dataset = MyDataSet(test_audios, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

In [36]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(1024, 24)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|████                                                                                                                                                                                                         | 1/50 [00:03<02:42,  3.32s/it]

epoch:1, train accu:0.2282, train loss:1.79


  4%|████████▏                                                                                                                                                                                                    | 2/50 [00:06<02:34,  3.21s/it]

epoch:2, train accu:0.2936, train loss:1.66


  6%|████████████▎                                                                                                                                                                                                | 3/50 [00:09<02:29,  3.18s/it]

...

epoch:48, train accu:0.9535, train loss:0.14


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 49/50 [02:38<00:03,  3.19s/it]

epoch:49, train accu:0.9709, train loss:0.11


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:41<00:00,  3.23s/it]

epoch:50, train accu:0.9826, train loss:0.06


### Model Test

In [37]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.7413793103448276


In [38]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[ 0.1156],
         [ 0.0656],
         [ 0.0633],
         [-0.2668],
         [-0.4016],
         [-0.2134],
         [-0.3325],
         [-0.2353],
         [ 0.0787],
         [-0.1615],
         [ 0.1373],
         [ 0.2209],
         [ 0.1435],
         [-0.0522],
         [ 0.1635],
         [-0.0688],
         [-0.0029],
         [ 0.0599],
         [-0.1646],
         [-0.0916],
         [ 0.0985],
         [ 0.0864],
         [ 0.0154],
         [-0.0357]]], device='cuda:0')
aggr.bias tensor([0.0273], device='cuda:0')
embed.weight tensor([[-0.0095, -0.0095,  0.0412,  ...,  0.0129, -0.0263, -0.1223],
        [-0.0056, -0.0213, -0.0185,  ..., -0.0636, -0.0060,  0.0211],
        [ 0.0030,  0.0115,  0.0145,  ...,  0.0168,  0.0214, -0.0301],
        ...,
        [ 0.0117,  0.0160, -0.0048,  ..., -0.0592, -0.0623,  0.0354],
        [-0.0134, -0.0210,  0.0207,  ..., -0.0141, -0.0122, -0.0183],
        [-0.0784, -0.0109, -0.0090,  ..., -0.0176,  0.0402, -0.0975]]

In [39]:
model_path = main_path + f'/models/hubertlarge/holdout_{holdout}.pth'

torch.save({'epoch':epochs,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            model_path)